# Pairs Trading- finding pairs based on Clustering

In this case study, we will use clustering methods to select pairs for a pairs trading
strategy.

<a id='0'></a>
# 1. Problem Definition

Our goal in this case study is to perform clustering analysis on the stocks of S&P500
and come up with pairs for a pairs trading strategy.


The data of the stocks of S&P 500, obtained using pandas_datareader from yahoo
finance. It includes price data from 2018 onwards.

<a id='1'></a>
# 2. Getting Started- Loading the data and python packages

<a id='1.1'></a>
## 2.1. Loading the python packages

In [1]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pandas_datareader as dr
import pickle

# Import Model Packages 
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold

# Other Helper Packages and functions
import matplotlib.ticker as ticker
from itertools import cycle

import yahoo_fin.stock_info as si
import yfinance as yf

from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autotime

time: 0 ns (started: 2022-10-16 10:04:50 +08:00)


<a id='1.2'></a>
## 2.2. Loading the Data

In [2]:
# The dataset was obtained from Kaggle and imported
df = pd.read_csv('../bigdata/stockprices.csv')

time: 1.19 s (started: 2022-10-16 10:04:50 +08:00)


In [3]:
df.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04 00:00:00-05:00,MMM,59.318886,83.019997,83.449997,82.669998,83.089996,3043700.0
1,2010-01-05 00:00:00-05:00,MMM,58.947342,82.500000,83.230003,81.699997,82.800003,2847000.0
2,2010-01-06 00:00:00-05:00,MMM,59.783295,83.669998,84.599998,83.510002,83.879997,5268500.0
3,2010-01-07 00:00:00-05:00,MMM,59.826176,83.730003,83.760002,82.120003,83.320000,4470100.0
4,2010-01-08 00:00:00-05:00,MMM,60.247749,84.320000,84.320000,83.300003,83.690002,3405800.0


time: 16 ms (started: 2022-10-16 10:04:51 +08:00)


In [4]:
len(df)

1619660

time: 0 ns (started: 2022-10-16 10:04:51 +08:00)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1619660 entries, 0 to 1619659
Data columns (total 8 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Date       1619660 non-null  object 
 1   Symbol     1619660 non-null  object 
 2   Adj Close  1547214 non-null  float64
 3   Close      1547214 non-null  float64
 4   High       1547214 non-null  float64
 5   Low        1547214 non-null  float64
 6   Open       1547214 non-null  float64
 7   Volume     1547214 non-null  float64
dtypes: float64(6), object(2)
memory usage: 98.9+ MB
time: 78 ms (started: 2022-10-16 10:04:51 +08:00)


In [6]:
# Define start date and end date for the data
start_date = '2015-01-03'
end_date = '2019-12-31'

df_filter_dates = df[(df.Date>=start_date) & (df.Date<=end_date)]

# Additionally keep two more unseen datasets for testing later
df_2020 = df[(df.Date>='2020-01-01') & (df.Date<='2020-12-31')]
df_2021 = df[(df.Date>='2021-01-01') & (df.Date<='2021-12-31')]
df_2022 = df[(df.Date>='2022-01-01') & (df.Date<='2022-12-31')]

time: 422 ms (started: 2022-10-16 10:04:51 +08:00)


In [7]:
# Saved to filtered file to csv
df_filter_dates.to_csv('./data/prices_2015_2019.csv')

time: 3.83 s (started: 2022-10-16 10:04:52 +08:00)


In [8]:
df_filter_dates.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
1260,2015-01-05 00:00:00-05:00,MMM,129.675308,160.360001,163.639999,160.080002,163.000000,3692900.0
1261,2015-01-06 00:00:00-05:00,MMM,128.292511,158.649994,161.369995,157.740005,160.820007,3532000.0
1262,2015-01-07 00:00:00-05:00,MMM,129.222473,159.800003,160.279999,158.940002,159.899994,3081300.0
1263,2015-01-08 00:00:00-05:00,MMM,132.319626,163.630005,163.690002,160.520004,160.649994,3142900.0
1264,2015-01-09 00:00:00-05:00,MMM,130.694183,161.619995,164.000000,161.270004,163.850006,2378600.0


time: 16 ms (started: 2022-10-16 10:04:55 +08:00)


In [9]:
# Pivot and transform the table
dataset = pd.pivot_table(df_filter_dates, index='Date', columns='Symbol', values='Adj Close')

time: 375 ms (started: 2022-10-16 10:04:55 +08:00)


In [10]:
dataset.index

Index(['2015-01-05 00:00:00-05:00', '2015-01-06 00:00:00-05:00',
       '2015-01-07 00:00:00-05:00', '2015-01-08 00:00:00-05:00',
       '2015-01-09 00:00:00-05:00', '2015-01-12 00:00:00-05:00',
       '2015-01-13 00:00:00-05:00', '2015-01-14 00:00:00-05:00',
       '2015-01-15 00:00:00-05:00', '2015-01-16 00:00:00-05:00',
       ...
       '2019-12-16 00:00:00-05:00', '2019-12-17 00:00:00-05:00',
       '2019-12-18 00:00:00-05:00', '2019-12-19 00:00:00-05:00',
       '2019-12-20 00:00:00-05:00', '2019-12-23 00:00:00-05:00',
       '2019-12-24 00:00:00-05:00', '2019-12-26 00:00:00-05:00',
       '2019-12-27 00:00:00-05:00', '2019-12-30 00:00:00-05:00'],
      dtype='object', name='Date', length=1256)

time: 0 ns (started: 2022-10-16 10:04:56 +08:00)


In [11]:
# Format the date column
dataset.index = pd.to_datetime(dataset.index, utc=True)

time: 0 ns (started: 2022-10-16 10:04:56 +08:00)


In [12]:
dataset.index = pd.to_datetime(dataset.index).strftime('%Y-%m-%d')

time: 0 ns (started: 2022-10-16 10:04:56 +08:00)


<a id='2'></a>
# 3. Exploratory Data Analysis

<a id='2.1'></a>
## 3.1. Descriptive Statistics

In [13]:
# Shape of dataset
dataset.shape

(1256, 499)

time: 0 ns (started: 2022-10-16 10:04:56 +08:00)


In [14]:
# peek at data
set_option('display.width', 100)
dataset.head(5)

Symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,37.420185,51.051483,151.336792,24.120045,46.710945,79.661964,37.070000,39.125034,76.426567,71.980003,...,127.430626,29.178459,64.219269,49.259911,32.530121,44.445164,107.125671,76.339996,24.042185,41.035954
2015-01-06,36.837265,50.255585,151.230377,24.122320,46.479752,80.097168,36.130001,38.680729,75.875282,70.529999,...,124.712402,29.333715,63.877869,48.648579,32.338821,43.899632,106.217575,75.790001,23.123941,40.635601
2015-01-07,37.326168,50.227158,154.480148,24.460564,48.358299,81.695938,37.279999,38.994354,77.467850,71.110001,...,128.374954,29.587006,64.525108,49.909447,32.593887,45.354382,108.859299,77.720001,23.344675,41.474419
2015-01-08,38.445004,50.843037,155.834167,25.400398,48.864059,81.882439,38.959999,39.795853,78.649185,72.919998,...,129.919449,29.930189,65.599113,51.141659,32.839859,46.144466,110.015007,79.379997,23.671360,42.113091
2015-01-09,38.162949,49.289127,155.041122,25.427635,47.527401,82.601891,38.410000,39.377686,78.561676,71.839996,...,131.543274,29.562494,65.506653,50.396595,32.220402,46.062954,108.428177,80.540001,23.009155,42.179813


time: 0 ns (started: 2022-10-16 10:04:56 +08:00)


In [15]:
# describe data
set_option('precision', 3)
dataset.describe()

Symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
count,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,...,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000,1256.000
mean,55.006,39.638,143.622,36.906,59.805,82.091,182.990,52.047,127.330,167.241,...,111.760,40.805,62.954,53.025,54.490,70.015,109.944,122.043,36.427,68.409
std,14.512,7.182,23.422,11.335,14.624,8.115,107.157,15.550,33.608,79.427,...,30.902,9.438,3.668,7.393,16.691,18.610,11.452,53.268,9.894,25.690
min,31.622,24.281,77.127,21.099,36.022,62.801,36.130,32.535,73.530,69.990,...,46.518,26.588,50.110,33.568,27.965,42.096,84.815,46.930,17.704,37.846
25%,39.922,33.877,136.224,26.777,47.497,76.607,97.265,39.348,100.986,93.993,...,89.535,34.143,60.485,49.051,38.141,55.215,103.562,80.683,25.753,46.105
50%,57.711,39.597,149.835,36.048,56.181,81.444,142.980,45.068,117.823,143.605,...,111.691,39.919,63.094,54.760,51.807,68.095,109.340,105.910,38.932,60.530
75%,66.881,45.370,158.723,45.070,73.447,85.989,265.538,64.727,152.798,249.120,...,131.382,45.372,65.577,58.498,71.192,82.795,116.975,163.580,45.054,86.562
max,84.129,56.989,194.023,72.026,99.695,102.871,449.750,85.388,205.783,331.200,...,190.356,62.595,71.204,66.581,82.366,114.009,144.435,258.280,53.566,131.554


time: 500 ms (started: 2022-10-16 10:04:56 +08:00)


<a id='3'></a>
## 4. Data Preparation

<a id='3.1'></a>
## 4.1. Data Cleaning
We check for the NAs in the rows, either drop them or fill them with the mean of the column.

In [16]:
#Checking for any null values and removing the null values'''
print('Null Values =', dataset.isnull().values.any())

Null Values = True
time: 0 ns (started: 2022-10-16 10:04:56 +08:00)


In [17]:
# Getting rid of columns with more than 30% missing values
missing_fractions = dataset.isnull().mean().sort_values(ascending=False)

missing_fractions.head(10)

drop_list = sorted(list(missing_fractions[missing_fractions > 0.2].index))

dataset.drop(labels=drop_list, axis=1, inplace=True)
dataset.shape

(1256, 487)

time: 15 ms (started: 2022-10-16 10:04:56 +08:00)


Given that there are null values drop the rown contianing the null values.

In [18]:
# Fill the missing values with the last value available in the dataset. 
dataset=dataset.fillna(method='ffill')
dataset.head(2)

Symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,37.420,51.051,151.337,24.120,46.711,79.662,37.07,39.125,76.427,71.98,...,127.431,29.178,64.219,49.260,32.530,44.445,107.126,76.34,24.042,41.036
2015-01-06,36.837,50.256,151.230,24.122,46.480,80.097,36.13,38.681,75.875,70.53,...,124.712,29.334,63.878,48.649,32.339,43.900,106.218,75.79,23.124,40.636


time: 16 ms (started: 2022-10-16 10:04:56 +08:00)


In [19]:
# To fill up any null values left over
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="mean")
dataset = pd.DataFrame(imp.fit_transform(dataset), columns=dataset.columns, index=dataset.index)

time: 15 ms (started: 2022-10-16 10:04:56 +08:00)


In [20]:
pickle.dump(dataset, open('./data/01_pricedate_2015_2019.pkl', 'wb'))

time: 16 ms (started: 2022-10-16 10:04:57 +08:00)
